In [88]:
import pandas as pd
import numpy as np

In [89]:
link_df = pd.read_csv("links.csv", index_col=0)
link_df.head()

,from,to,value,from_id,to_id
0,慶應義塾大学,研究,0.025952,4,1405
1,慶應義塾大学,学部,0.022282,4,439
2,慶應義塾大学,月,0.020959,4,2603
3,慶應義塾大学,学生,0.020782,4,1603
4,慶應義塾大学,情報,0.019950,4,2291


In [90]:
node_df = pd.read_csv("nodes.csv", index_col=0)
node_df.head(6)

,query,from_ut,from_ky,from_wa,from_ko,from_me,shortest_step,label_ut,label_ky,label_wa,label_ko,label_me,index
0,東京大学,0,2,2,2,2,0,True,False,False,False,False,0
1,京都大学,2,0,2,2,2,0,False,True,False,False,False,1
2,明治大学,2,2,2,2,0,0,False,False,False,False,True,2
3,早稲田大学,2,2,0,2,2,0,False,False,True,False,False,3
4,慶應義塾大学,2,2,2,0,2,0,False,False,False,True,False,4
5,PDF,2,2,3,3,2,2,True,True,False,False,True,5


In [91]:
node_df[node_df.shortest_step == 0]

,query,from_ut,from_ky,from_wa,from_ko,from_me,shortest_step,label_ut,label_ky,label_wa,label_ko,label_me,index
0,東京大学,0,2,2,2,2,0,True,False,False,False,False,0
1,京都大学,2,0,2,2,2,0,False,True,False,False,False,1
2,明治大学,2,2,2,2,0,0,False,False,False,False,True,2
3,早稲田大学,2,2,0,2,2,0,False,False,True,False,False,3
4,慶應義塾大学,2,2,2,0,2,0,False,False,False,True,False,4


In [92]:
node_df.columns

Index(['query', 'from_ut', 'from_ky', 'from_wa', 'from_ko', 'from_me',
       'shortest_step', 'label_ut', 'label_ky', 'label_wa', 'label_ko',
       'label_me', 'index'],
      dtype='object')

In [93]:
node_ids = node_df['index'].values

In [94]:
nodes_data = node_df[['from_ut', 'from_ky', 'from_wa', 'from_ko', 'from_me']].values

In [95]:
np.where(nodes_data[5] == nodes_data[5].min())

(array([0, 1, 4]),)

In [96]:
np.where(nodes_data[5] == nodes_data[5].min())[0]

array([0, 1, 4])

In [97]:
len(np.where(nodes_data[5] == nodes_data[5].min())[0])

3

In [98]:
nodes_data[5].argmin()

0

In [99]:
nearest_univ_nodes = [np.where(node_data == node_data.min())[0] for node_data in nodes_data[5:]]

In [100]:
nearest_univ_nodes

[array([0, 1, 4]),
 array([0, 2, 3, 4]),
 array([1]),
 array([0, 1, 2, 3, 4]),
 array([0, 3]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3]),
 array([0, 1, 2, 3, 4]),
 array([4]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([2, 3]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 2, 3]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([3]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 1, 2, 3, 4]),
 array([0, 2, 3

In [101]:
univs_positions = np.zeros((5, 3))
univs_positions[0] = np.array([-1,1,1])
univs_positions[1] = np.array([-1,-1,-1])
univs_positions[2] = np.array([1,1,-1])
univs_positions[3] = np.array([1,-1,1])
univs_positions[4] = np.array([-1,1,-1])

In [119]:
from numpy.random import *
positions = []
for nearest_univ_node in nearest_univ_nodes:
    if len(nearest_univ_node) == 1:
        if nearest_univ_node[0] == 0:
            p = np.array([uniform(-3, -2, 1)[0], uniform(2, 3, 1)[0], uniform(2, 3, 1)[0]])
        elif nearest_univ_node[0] == 1:
            p = uniform(-3, -2, 3)
        elif nearest_univ_node[0] == 2:
            p = np.array([uniform(2, 3, 1)[0], uniform(-3, -2, 1)[0], uniform(2, 3, 1)[0]])
        elif nearest_univ_node[0] == 3:
            p = np.array([uniform(-3, -2, 1)[0], uniform(2, 3, 1)[0], uniform(-3, -2, 1)[0]])
        elif nearest_univ_node[0] == 4:
            p = np.array([uniform(2, 3, 1)[0], uniform(2, 3, 1)[0], uniform(-3, -2, 1)[0]])
    elif len(nearest_univ_node) == 2:
        zeros = np.where(univs_positions[nearest_univ_node[0]] + univs_positions[nearest_univ_node[1]] == 0)[0]
        p = [0, 0, 0]
        for i in range(3):
            if i in zeros:
                p[i] = uniform(-1, 1, 1)[0]
            else:
                univ_p = univs_positions[nearest_univ_node[0]][i]
                if univ_p == -1:
                    p[i] = uniform(-1.6, -1.1, 1)[0]
                else:
                    p[i] = uniform(1.1, 1.6, 1)[0]
        p = np.array(p)
    else:
        p = uniform(-1, 1, 3)
    positions.append(p)
not_univ_nodes_p = np.array(positions)

In [120]:
start_positions = np.vstack((univs_positions, not_univ_nodes_p))

In [121]:
start_positions[0:10]

array([[-1.        ,  1.        ,  1.        ],
       [-1.        , -1.        , -1.        ],
       [ 1.        ,  1.        , -1.        ],
       [ 1.        , -1.        ,  1.        ],
       [-1.        ,  1.        , -1.        ],
       [-0.9356269 , -0.75814837,  0.47083876],
       [-0.01232959, -0.27707602,  0.33899931],
       [-2.89367532, -2.34240341, -2.26735101],
       [-0.11028156, -0.06444975,  0.38971318],
       [ 0.34373715, -0.50799429,  1.38304753]])

In [122]:
from_ids =  np.array(link_df.from_id)
to_ids = np.array(link_df.to_id)
distances = np.array(node_df.shortest_step)
# from numpy.random import *
# start_positions = uniform(-2,2, (len(node_ids), 3))
# start_positions[0] = np.array([-1,1,1])
# start_positions[1] = np.array([-1,-1,-1])
# start_positions[2] = np.array([1,1,-1])
# start_positions[3] = np.array([1,-1,1])
# start_positions[4] = np.array([-1,1,-1])
# import copy
# positions = copy.deepcopy(start_positions)

In [123]:
# 重心法
import copy
positions = copy.deepcopy(start_positions)
for i in range(500): 
    Fs = [0]* len(node_ids)
    for node_id in node_ids:
        if node_id not in [0, 1, 2, 3, 4]:
            P = positions[node_id]
            F = np.array([0, 0, 0])
            linked_node_ids = to_ids[np.where(from_ids == node_id)]
#             print(P)
            for node_i_id in linked_node_ids:
                D_i = float(distances[node_i_id] + 1)
                P_i = positions[node_i_id]
                x_i = (P_i - P) - ((P_i - P) / np.linalg.norm(P_i - P)) # バネの伸びベクトル(バネのもとの長さは1)
                F_i = x_i * (1/D_i)  # Nに働く力ベクトルを算出(バネ定数はD_iの逆数)
                F = F + F_i
            Fs[node_id] =F / len(linked_node_ids)
    for node_id in node_ids:
        positions[node_id] = positions[node_id] + Fs[node_id]
    if i % 10 == 0:
        print(np.sum(np.absolute(np.array(Fs[5:]))))

944.7120296006326
95.84850706368303
44.05068985555636
28.492644476961793
21.297237712628515
17.1597368298754
14.439854610283259
12.52131072876255
11.137024310612471
10.126056993148046
9.358525987831023
8.725640999813905
8.175852929648764
7.698902627168447
7.258344856261669
6.818407308562086
6.373952826994691
5.961032235487667
5.589146017321811
5.251550266853744
4.940147882830087
4.656384240170954
4.3953208674838695
4.1556857315717135
3.93873306275197
3.745361727449753
3.5694175999094653
3.408217472432021
3.266271937377041
3.1420993673335906
3.0330783309174683
2.933168631827003
2.842839606072921
2.7623413971309683
2.6916457987570364
2.6284334512736276
2.568693867700286
2.5117182929395545
2.4542774240150265
2.3969291201351126
2.341473390286981
2.2898062097053975
2.2455090787455934
2.2072976209899435
2.17347151223726
2.1439972150941116
2.1148385466410735
2.081801169002912
2.039130443691369
1.9884264496730208


In [125]:
# 万有引力法
import copy
positions = copy.deepcopy(start_positions)
for i in range(500): 
    Fs = [0]* len(node_ids)
    for node_id in node_ids:
        if node_id not in [0, 1, 2, 3, 4]:
            P = positions[node_id]
            F = np.array([0, 0, 0])
            linked_node_ids = to_ids[np.where(from_ids == node_id)]
#             print(P)
            for node_i_id in linked_node_ids:
                D_i = float(distances[node_i_id] + 1)
                P_i = positions[node_i_id]
                x_i = (P_i - P) - ((P_i - P) / np.linalg.norm(P_i - P)) # バネの伸びベクトル(バネのもとの長さは1)
                F_i = x_i * (1/D_i) ** 2  # Nに働く力ベクトルを算出(バネ定数はD_iの逆数)
                F = F + F_i
            Fs[node_id] =F / len(linked_node_ids)
    for node_id in node_ids:
        positions[node_id] = positions[node_id] + Fs[node_id]
    if i % 10 == 0:
        print(np.sum(np.absolute(np.array(Fs[5:]))))

341.28072375043814
99.90562406105121
53.93262656226443
36.02587281282808
26.44182829636983
20.43799104614871
16.553520732746176
13.926695925192707
12.049846240805875
10.644957479993913
9.53567046326202
8.640042613034064
7.900516649290599
7.264944393239552
6.72389443567652
6.257572424640176
5.854758947561298
5.502473743655401
5.192251132854796
4.918490168500768
4.671876049394606
4.443599491706788
4.234089633042484
4.0395811490331095
3.8602044355777756
3.6934196316902796
3.5390310852680584
3.393365218121219
3.2570038577168776
3.1275446434285934
3.006369905977177
2.8942503949044807
2.790525433646076
2.694377994166578
2.6059842310874313
2.524087927541565
2.4472063379627014
2.3757324350899163
2.3092328330207432
2.2470531261190207
2.1883751826399265
2.1328669889354233
2.081146110468398
2.0326855301068742
1.9864564724570133
1.9428180377679636
1.9014658513304352
1.8615907183338922
1.8232111460048872
1.7862511775477425


In [82]:
# takeuchi
import copy
positions = copy.deepcopy(start_positions)
for i in range(500): 
    Fs = [0]* len(node_ids)
    for node_id in node_ids:
        if node_id not in [0, 1, 2, 3, 4]:
            P = positions[node_id]
            F = np.array([0, 0, 0])
            linked_node_ids = to_ids[np.where(from_ids == node_id)]
            for node_i_id in linked_node_ids:
                D_i = float(distances[node_i_id] + 1)
                P_i = positions[node_i_id]
                x_i = (P_i - P) - ((P_i - P) / np.linalg.norm(P_i - P)) # バネの伸びベクトル(バネのもとの長さは1)
                F_i = x_i * (1/D_i)  # Nに働く力ベクトルを算出(バネ定数はD_iの逆数)
                F = F + F_i
            Fs[node_id] = F * D_i * 0.001
    for node_id in node_ids:
        positions[node_id] = positions[node_id] + Fs[node_id]
    if i % 10 == 0:
        print(np.sum(np.absolute(np.array(Fs[5:]))))

/Users/bearshu11/.anyenv/envs/pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  


nan


KeyboardInterrupt: 

In [126]:
pd.DataFrame(positions, columns=['x','y','z']).to_csv('banyuinryoku.csv')
# pd.DataFrame(positions, columns=['x','y','z']).to_csv('takeuchi2.csv')

In [179]:
start_positions

array([[-1.        ,  1.        ,  1.        ],
       [-1.        , -1.        , -1.        ],
       [ 1.        ,  1.        , -1.        ],
       ...,
       [ 0.6928154 ,  1.68818213, -0.9419346 ],
       [ 0.66579461, -0.43664928, -1.49406485],
       [ 1.44154426,  1.92945304, -0.58298092]])

In [180]:
positions

array([[-1.        ,  1.        ,  1.        ],
       [-1.        , -1.        , -1.        ],
       [ 1.        ,  1.        , -1.        ],
       ...,
       [ 0.36739009,  1.24925322, -0.72338481],
       [ 0.45182163, -0.13944754, -0.7450748 ],
       [ 0.05837693,  0.54831132, -0.02332937]])